<a href="https://colab.research.google.com/github/dmlc/gluon-cv/blob/onnx/scripts/onnx/notebooks/pose/simple_pose_resnet18_v1b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install --upgrade onnxruntime

In [ ]:

import numpy as np
import onnxruntime as rt
import urllib.request
import os.path
from PIL import Image
    

In [ ]:

def fetch_detect_model():
    if not os.path.isfile("yolo3_mobilenet1.0_coco.onnx"):
        urllib.request.urlretrieve("https://apache-mxnet.s3-us-west-2.amazonaws.com/onnx/models/gluoncv-yolo3_mobilenet1.0_coco-115299e3.onnx", filename="yolo3_mobilenet1.0_coco.onnx")
    return 'yolo3_mobilenet1.0_coco.onnx'
    
def fetch_model():
    if not os.path.isfile("simple_pose_resnet18_v1b.onnx"):
        urllib.request.urlretrieve("https://apache-mxnet.s3-us-west-2.amazonaws.com/onnx/models/gluoncv-simple_pose_resnet18_v1b-cbd4ac31.onnx", filename="simple_pose_resnet18_v1b.onnx")
    return "simple_pose_resnet18_v1b.onnx"

def prepare_img(img_path, input_shape):
    # input_shape: BHWC
    height, width = input_shape[1], input_shape[2]
    img = Image.open(img_path).convert('RGB')
    img = img.resize((width, height))
    img = np.asarray(img)
    plt_img = img
    img = np.expand_dims(img, axis=0).astype('float32')

    return plt_img, img
    

**Make sure to replace the image you want to use**

In [ ]:

detector = fetch_detect_model()
model = fetch_model()
img_path = 'Your image'
plt_img, img = prepare_img(img_path, (1, 512, 512, 3))
    

In [ ]:

detector_session = rt.InferenceSession(detector, None)
detector_input_name = detector_session.get_inputs()[0].name
    

In [ ]:

class_IDs, scores, bounding_boxs = detector_session.run([], {detector_input_name: img})



(Optional) We use mxnet and gluoncv to process the detector result.

Feel free to process the result your own way


In [ ]:
!pip3 install --upgrade mxnet gluoncv

In [ ]:

import mxnet as mx
from gluoncv.data.transforms.pose import detector_to_simple_pose

class_IDs = mx.nd.array(class_IDs)
scores = mx.nd.array(scores)
bounding_boxs = mx.nd.array(bounding_boxs)
# Exported model has preprocess layer, so we pass mean=0 and std=1 to disable preprocess
# when converting to pose input
pose_input, upscale_bbox = detector_to_simple_pose(plt_img, class_IDs, scores, bounding_boxs,
                                                   mean=(0,0,0), std=(1,1,1), output_shape=(256, 192))
pose_input = np.transpose(pose_input, (0,2,3,1))
pose_input *= 255  
    

In [ ]:

# Create a onnx inference session and get the input name
pose_session = rt.InferenceSession(model, None)
pose_input_name = pose_session.get_inputs()[0].name   
    

In [ ]:

predicted_heatmap = np.concatenate([pose_session.run([], {pose_input_name: pi.expand_dims(0).asnumpy()})[0] 
                                    for pi in pose_input])
    


(Optional) We use mxnet and gluoncv to visualize the  result.

Feel free to visualize the result your own way


In [ ]:

import mxnet as mx
from gluoncv import utils
from matplotlib import pyplot as plt
from gluoncv.data.transforms.pose import heatmap_to_coord

predicted_heatmap = mx.nd.array(predicted_heatmap)
pred_coords, confidence = heatmap_to_coord(predicted_heatmap, upscale_bbox)
ax = utils.viz.plot_keypoints(plt_img, pred_coords, confidence,
                              class_IDs, bounding_boxs, scores,
                              box_thresh=0.5, keypoint_thresh=0.2)
plt.show()
    